In [1]:
from grpo import *


/u/jgottesman/.conda/envs/grposloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-28 18:01:22 [__init__.py:239] Automatically detected platform cuda.


2025-03-28 18:01:22,289	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
dataset = get_TSP_questions()

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# if "Llama" in model_name:
#     output_dir = "outputs/Llama-1B-GRPO"
#     run_name = "Llama-1B-GRPO-gsm8k"
# else:
#     output_dir="outputs/Qwen-1.5B-GRPO"
#     run_name="Qwen-1.5B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir="outputs/test",
    run_name="test_run_1",
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_generations=8,
    max_prompt_length=256,
    max_completion_length=786,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    # report_to="wandb",
    log_on_each_node=False,
)
peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
)


from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        optimal_solution_reward,
        improvement_reward_func],
    args=training_args,
    train_dataset=dataset,
    #peft_config=peft_config
)
trainer.train()

/tmp/ipykernel_2764676/2825987000.py:48: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Standard import failed for UnslothPRMTrainer: No module named 'UnslothPRMTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-1b-instruct-bnb-4bit with actual GPU utilization = 59.31%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.38 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 288.
Unsloth: vLL

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.56it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.55it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.07it/s]


INFO 03-28 18:01:37 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 03-28 18:01:38 [model_runner.py:1146] Model loading took 1.0372 GB and 0.919161 seconds
INFO 03-28 18:01:39 [worker.py:267] Memory profiling takes 1.01 seconds
INFO 03-28 18:01:39 [worker.py:267] the current vLLM instance can use total_gpu_memory (39.38GiB) x gpu_memory_utilization (0.59) = 23.36GiB
INFO 03-28 18:01:39 [worker.py:267] model weights take 1.04GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.32GiB; the rest of the memory reserved for KV Cache is 20.90GiB.
INFO 03-28 18:01:39 [executor_base.py:111] # cuda blocks: 42810, # CPU blocks: 12288
INFO 03-28 18:01:39 [executor_base.py:116] Maximum concurrency for 1024 tokens per request: 668.91x
INFO 03-28 18:01:43 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider dec

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:21<00:00,  1.79it/s]

INFO 03-28 18:02:05 [model_runner.py:1570] Graph capturing finished in 22 secs, took 0.40 GiB
INFO 03-28 18:02:05 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 27.60 seconds
